In [1]:
import gc
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras import optimizers
from keras import Sequential

C:\Users\Bailey\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
train_df = pd.read_csv('train_extra.csv')
test_df = pd.read_csv('test.csv')
print("load done")

load done


In [3]:
def remove_unknown_tag(col, train_df = train_df, test_df = test_df):
    test_df.loc[~test_df[col].isin(train_df[col]),col] = 9999999

In [4]:
def remove_lowfreq_tag(col, train_df = train_df,test_df = test_df, N=3):
    topN_address_list = train_df[col].value_counts()
    #print(topN_address_list)
    topN_address_list = topN_address_list[topN_address_list <= N]
    topN_address_list = topN_address_list.index
    remove_list = train_df.loc[train_df[col].isin(topN_address_list), col]
    print('remove:',len(remove_list))
    print('reserve',len(train_df) - len(remove_list))
    remove_list = 9999998
    test_df.loc[test_df[col].isin(topN_address_list), col] = 9999998

In [10]:
def preprocess(df):
    df['time_t'] = df.click_time.str[11:13] #+ df.click_time.str[14:16]
    #df['time_t'] = df['time_t'].astype(datetime)

In [6]:
for i in ['app','ip','device','os','channel']:
    remove_unknown_tag(i)

In [7]:
for i,j in zip(['app','ip','device','os','channel'],[5,4,4,5,5]):
    remove_lowfreq_tag(i,N=j)

remove: 218
reserve 119782
remove: 63329
reserve 56671
remove: 600
reserve 119400
remove: 125
reserve 119875
remove: 39
reserve 119961


In [8]:
def process_lable_encoder(col, train_df = train_df, test_df = test_df):
    le = LabelEncoder()
    le.fit(np.hstack([train_df[col], test_df[col]]))
    train_df[col] = le.transform(train_df[col])
    test_df[col] = le.transform(test_df[col])
    
coding_list = ['ip','app','device','os','channel','time_t']

In [11]:
preprocess(train_df)
preprocess(test_df)
print("preprocessing done")

preprocessing done


In [13]:
for i in coding_list:
    process_lable_encoder(i)
    
print("label encoding done")

label encoding done


In [35]:
#train_df1 = train_df.drop(['click_time','attributed_time'],axis=1)
test_df1 = test_df.drop(['click_time','click_id'],axis=1)

In [34]:
train_df1.head()
test_df1.head()

,click_id,ip,app,device,os,channel,time_t
0,0,46326,9,1,3,15,4
1,1,22229,9,1,3,149,4
2,2,13365,21,1,19,32,4
3,3,14485,15,1,13,18,4
4,4,46326,12,1,13,90,4


In [26]:
df_train, df_test = train_test_split(train_df1, test_size = 0.25, random_state = 19)

# train_df.is_attributed.sum()/train_df.shape[0]
# test_df.is_attributed.sum()/test_df.shape[0]
                                     
train_y = df_train.is_attributed
test_y = df_test.is_attributed

train_x = df_train.drop(['is_attributed'], axis = 1)
test_x = df_test.drop(['is_attributed'], axis = 1)

train_x = np.array(train_x)
test_x = np.array(test_x)

train_y = np.array(train_y)
test_y = np.array(test_y)

In [27]:
adam = optimizers.adam(lr = 0.001, decay = 0.0000001)

model = Sequential()
model.add(Dense(48, input_dim=train_x.shape[1],kernel_initializer='normal',activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(24,activation = "tanh"))
model.add(Dropout(0.3))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.compile(loss="binary_crossentropy",optimizer='adam')

history = model.fit(train_x,train_y, validation_split=0.2, epochs=3, batch_size=64)

Train on 72000 samples, validate on 18000 samples
Epoch 1/3
72000/72000 [==============================] - 7s 95us/step - loss: 0.4804 - val_loss: 0.4527
Epoch 2/3
72000/72000 [==============================] - 5s 70us/step - loss: 0.4582 - val_loss: 0.4534
Epoch 3/3
72000/72000 [==============================] - 5s 69us/step - loss: 0.4558 - val_loss: 0.4532


In [36]:
predictions_NN_prob = model.predict(test_df1)
predictions_NN_prob = predictions_NN_prob[:,0]

In [38]:
test_df1['is_attributed'] = predictions_NN_prob
df1 = test_df1
df1 = df1.drop(['app','device','os','channel','time_t'],axis=1)
df1['click_id'] = df1.index
df1 = df1[['click_id','is_attributed']]

In [39]:
df1.to_csv("NNwithoutembedding.csv", index = False)